In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.3426, -0.0367,  0.1814, -0.0362, -0.0223, -0.2101, -0.0265, -0.0447,
          0.0129, -0.0741],
        [ 0.1603,  0.0215,  0.2318,  0.0380,  0.0109, -0.1323, -0.0206, -0.1539,
          0.0761, -0.0094]], grad_fn=<AddmmBackward0>)

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # hidden layer
        self.out = nn.Linear(256, 10)  # output layer

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
net(X)

tensor([[ 0.0962, -0.1988, -0.1233, -0.0645, -0.1751, -0.0161,  0.0590, -0.0105,
         -0.1400, -0.0441],
        [ 0.0846, -0.1918, -0.0957, -0.0465, -0.1249,  0.0290,  0.0906, -0.0833,
         -0.1783, -0.1091]], grad_fn=<AddmmBackward0>)

In [8]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [9]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0813,  0.0457, -0.0659,  0.0627, -0.0049,  0.2393, -0.1321, -0.0184,
          0.0660, -0.1028],
        [ 0.0501, -0.0057,  0.0376,  0.1599,  0.1247,  0.2213, -0.2079, -0.0369,
         -0.0186, -0.0455]], grad_fn=<AddmmBackward0>)

In [10]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [14]:
net = FixedHiddenMLP()
net(X)

tensor(0.0223, grad_fn=<SumBackward0>)

In [16]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))


chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1937, grad_fn=<SumBackward0>)